# [Multi-AI-Foundry Agents with AutoGen 0.4](https://github.com/kinfey/MultiAIAgent/blob/main/04.AzureAIAgentWithAutoGen02.ipynb)

# Constants

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
env_or_file='./../config/models_list.json'
filter_dict = {
    'endpoint': 'https://mmoaiswc-01.openai.azure.com/',
    'deployment': 'gpt-4o-2024-08-06'
}

model_name =  filter_dict["deployment"] # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup
project_connection_string = os.environ["PROJECT_CONNECTION_STRING"]

print(f'Project Connection String: <...{project_connection_string[-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


# Helper functions
Inspired by [Migration Guide for v0.2 to v0.4](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/migration-guide.html)

In [2]:
def config_list_from_json(env_or_file, filter_dict):
    import json
    with open(env_or_file, 'r') as file:
        data = json.load(file)
    
    filtered_data = [
        item for item in data
        if item.get('endpoint') == filter_dict.get('endpoint') and item.get('deployment') == filter_dict.get('deployment')
    ]    
    return filtered_data


autogen_config = config_list_from_json(env_or_file, filter_dict)[0] # we take the first combination of model and endpoint

# beaware NOT to show the API KEY
print(f'AutoGen Configuration: {autogen_config["endpoint"]}, {autogen_config["deployment"]}, {autogen_config["api_version"]}, ...') 

AutoGen Configuration: https://mmoaiswc-01.openai.azure.com/, gpt-4o-2024-08-06, 2024-10-01-preview, ...


# Model client

In [27]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=autogen_config["endpoint"],
    api_key=autogen_config["api_key"],
    model = autogen_config["model"],
    azure_deployment = autogen_config["deployment"],
    api_version=autogen_config["api_version"],
    seed = 41,
    temperature = 0.1,
)

# model_client.dump_component() # be aware of the keys
print ('OpenAI endpoint: {model_client.dump_component().config["azure_endpoint"]},\nOpenAI Deployment: {model_client.dump_component().config["azure_deployment"]}')

OpenAI endpoint: {model_client.dump_component().config["azure_endpoint"]},
OpenAI Deployment: {model_client.dump_component().config["azure_deployment"]}


# [Assistant Agent definition: 0.2 vs 0.4](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/migration-guide.html#assistant-agent)

**In `Autogen v0.2`**, we created an assistant agent as follows:
```
llm_config = {
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API models configurations
        "temperature": 0,  # temperature for sampling
    }

student_agent = autogen.ConversableAgent (
    name = "Student_Agent",
    system_message = "You are a student willing to learn. You ask meaningful questions and are eager to learn more.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)
```

**In `Autogen v0.4`** it is similar, but we need to specify model_client instead of llm_config

```
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=autogen_config["endpoint"],
    api_key=autogen_config["api_key"],
    model = autogen_config["model"],
    azure_deployment = autogen_config["deployment"],
    api_version=autogen_config["api_version"],
    seed = 41,
    temperature = 0.1,
)
```

### OUTPUT
```
ComponentModel(provider='autogen_ext.models.openai.OpenAIChatCompletionClient', component_type='model', version=1, component_version=1, description=None, config={'seed': 42, 'temperature': 0.1, 'model': 'gpt-4o-2024-05-13', 'api_key': '***********', 'base_url': 'https://mmoaiswc-01.openai.azure.com/'})
```

# Define two equivalent Assistant Agents

In [4]:
# AGENT 1: THE STUDENT

from autogen_agentchat.agents import AssistantAgent

student_agent = AssistantAgent(
    name="student_agent",
    model_client=model_client,
    system_message="""
    You are a student willing to learn. You ask meaningful and precise follow-up questions and are eager to learn more.
    When someone answers a question of yours, you always make an example to be sure you correctly understood the answer.
    Wait for your example to be answered by your conterpart.
    As soon as the answer is reasonable complete, close the conversation saying 'STUDENT IS WILLING TO TERMINATE'.
    Be concise, no more than 100 words in your replies.
    """,
)

In [5]:
# AGENT 2: THE STUDENT

from autogen_agentchat.agents import AssistantAgent

teacher_agent = AssistantAgent(
    name="teacher_agent",
    model_client=model_client,
    system_message="""
    You are teacher expert in may disciplines, always happy to help students or other people willing to learn.
    Your approach is to challenge a little bit any opinions of others, as Socrate was willing to do much more.
    After one challenge, wait at least one reply from your conterpart.
    As soon as the answer is reasonable complete, wait for the counterpart reply and then close the conversation saying 'TEACHER IS WILLING TO TERMINATE'.
    Be concise, no more than 100 words in your replies.
    """,
)

# Termination Condition
It's a combination of text termination and max message termination, either of which will cause the chat to terminate.

In [6]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(20)

# Autogen Group Chats

## Peer-to-Peer Round Robin Group Chat

In [7]:
# The group chat will alternate between the assistant and the code executor.

from autogen_agentchat.teams import RoundRobinGroupChat
group_chat = RoundRobinGroupChat([teacher_agent, student_agent], termination_condition=termination)

stream = group_chat.run_stream(task="why does the water turns into ice under 0 celsius degrees?")

from autogen_agentchat.ui import Console
await Console(stream)

---------- user ----------
why does the water turns into ice under 0 celsius degrees?
---------- teacher_agent ----------
Water turns into ice under 0°C because the kinetic energy of water molecules decreases, causing them to move less and form a structured, crystalline lattice. This phase change is due to the hydrogen bonds between water molecules becoming more stable at lower temperatures. However, why do you think some substances don't freeze at 0°C?
---------- student_agent ----------
Some substances don't freeze at 0°C because their molecular structures and intermolecular forces differ from water. For example, alcohol has weaker hydrogen bonds and a lower freezing point. Is it correct to say that if I have a glass of pure water at -5°C, it will turn into ice because the temperature is below 0°C, allowing the water molecules to form a solid structure?
---------- teacher_agent ----------
Not necessarily. Pure water can remain liquid below 0°C in a state called supercooling, especial

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='why does the water turns into ice under 0 celsius degrees?', type='TextMessage'), TextMessage(source='teacher_agent', models_usage=RequestUsage(prompt_tokens=134, completion_tokens=64), content="Water turns into ice under 0°C because the kinetic energy of water molecules decreases, causing them to move less and form a structured, crystalline lattice. This phase change is due to the hydrogen bonds between water molecules becoming more stable at lower temperatures. However, why do you think some substances don't freeze at 0°C?", type='TextMessage'), TextMessage(source='student_agent', models_usage=RequestUsage(prompt_tokens=197, completion_tokens=77), content="Some substances don't freeze at 0°C because their molecular structures and intermolecular forces differ from water. For example, alcohol has weaker hydrogen bonds and a lower freezing point. Is it correct to say that if I have a glass of pure water at -5°C,

# Create AI Foundry Project Client

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool # <<<<<<<<<<<<<<< SPECIFIC FOR BING SEARCH
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=project_connection_string
)

project_client.scope

# Create the wrapper function for the BING AI Foundry Agent

In [ ]:
def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service...")
    
    # Retrieve the BING Connection already associated to the AI Foundry project
    bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
    
    # Build BingGroundingTool
    bing = BingGroundingTool(connection_id=bing_connection.id)

    # Create the Bing Agent
    agent = project_client.agents.create_agent(
        model=model_name,
        name="bing-agent",
        instructions="""
            You are a web search agent.
            Your only tool is search_tool - use it to find information.
            You make only one search call at a time.
            Once you have the resutls, you never do any elaboration of the obtained results.
            """,
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
        )
    print(f"Created BING agent, ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")
    
    # Add a user message to the thread
    message = project_client.agents.create_message(
        thread_id=thread.id, 
        role="user", 
        content=query, # "What is the top news today", "Quali sono i programmi TV stasera?"
    )
    print(f"Created message, ID: {message.id}")

    # Create and process assistant run in thread with tools
    run = project_client.agents.create_and_process_run\
        (thread_id=thread.id, assistant_id=agent.id)
    
    print(f"Run finished with status: {run.status}. Run id: {run.id}")
    
    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")
    
    if run.status == 'completed':    
        messages = project_client.agents.list_messages(thread_id=thread.id)
        # get the most recent message from the assistant
        last_msg = messages.get_last_text_message_by_sender("assistant")

    print(f"Number of annotation(s): {len(last_msg.text.annotations)}")

    a = 0
    for annotation in last_msg.text.annotations:
        a += 1
        print(f'- Annotation {a} of {len(last_msg.text.annotations)}.\n  - Text: {annotation["text"]}\n  - URL: {annotation["url_citation"]["url"]}')
        
    print (f"Deleting agent {agent.id}...")
    project_client.agents.delete_agent(agent.id)    
    
    if last_msg:
        print(f"\nLast Message: {last_msg.text.value}")

    return last_msg.text.value

In [ ]:
web_result = web_ai_agent("What is the top news today?")
web_result

In [ ]:
bing_search_agent.run("")

In [ ]:
from autogen_agentchat.agents import AssistantAgent

bing_search_agent = AssistantAgent(
    name="bing_search_agent",
    model_client=model_client,
    # tools=[web_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)

# Create AI Foundry Agent

In [ ]:
# Agent creation
# Notices that FileSearchTool as tool and tool_resources must be added or the assistant unable to search the file
agent = project_client.agents.create_agent(
    model=model_name,
    name="bing-agent",
    instructions="You are helpful assistant",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)

print(f"Created agent, ID: {agent}")

# Create the thread and attach a new message to it

In [ ]:
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id, 
    role="user", 
    content="What is the top news today?", # "What is the top news today", "Quali sono i programmi TV stasera?"
)
print(f"Created message: {message}")

# Run the agent syncrhonously

In [ ]:
%%time
# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run\
    (thread_id=thread.id, assistant_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Fetch messages from the thread after the agent run execution

In [ ]:
from azure.ai.projects.models import MessageTextContent, MessageImageFileContent

if run.status == 'completed':    
    messages = project_client.agents.list_messages(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages, starting with the most recent one:\n")
    i=0
    for m in messages.data:
        j = 0
        i += 1
        print(f"\n===== MESSAGE {messages_nr-i+1} =====")
        for c in m.content:
            j +=1
            if (type(c) is MessageImageFileContent):
                print(f"\nCONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
            elif (type(c) is MessageTextContent):
                print(f"\nCONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

In [ ]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

# Print annotations from the messages

In [ ]:
print(f"Number of annotation(s): {len(last_msg.text.annotations)}")

for annotation in last_msg.text.annotations:
    print(annotation["text"], annotation["url_citation"]["url"])

# Run Steps

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)

print(f"Nr of run step(s): {len(run_steps)}\n")
i=0
for rs in run_steps["data"]:
    i += 1
    print(f"Run step {i}: {rs}", '\n')

# START Teardown

In [ ]:
# Delete all agents

print(f"{len(project_client.agents.list_agents()['data'])} agent(s) will now be deleted")

i=0
for pca in project_client.agents.list_agents()['data']:
    i += 1
    project_client.agents.delete_agent(pca.id)
    print(f"\n{i} - Agent {pca.name} has been deleted")

# HIC SUNT LEONES